In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data=pd.read_excel("/content/jiffs_house_price_my_dataset_v1.xlsx")

In [5]:
data.head()

,land_size_sqm,house_size_sqm,no_of_rooms,no_of_bathrooms,large_living_room,parking_space,front_garden,swimming_pool,distance_to_school_km,wall_fence,house_age_or_renovated,water_front,distance_to_supermarket_km,property_value
0,201,177.0,3,1,0.0,1,1,0,3.3,1,10,0,6.8,167611
1,196,182.0,4,3,1.0,1,0,1,1.2,1,11,0,4.1,186945
2,198,182.0,4,4,1.0,1,0,1,5.9,0,20,0,2.1,154628
3,178,166.0,2,3,0.0,1,0,0,5.9,0,5,0,0.7,133099
4,183,165.0,3,1,1.0,1,0,0,3.8,1,8,0,0.7,158481


In [6]:
data.shape

(4954, 14)

In [7]:
data.dtypes

land_size_sqm                   int64
house_size_sqm                float64
no_of_rooms                     int64
no_of_bathrooms                 int64
large_living_room             float64
parking_space                   int64
front_garden                    int64
swimming_pool                   int64
distance_to_school_km         float64
wall_fence                      int64
house_age_or_renovated          int64
water_front                     int64
distance_to_supermarket_km    float64
property_value                  int64
dtype: object

In [8]:
data.tail()

,land_size_sqm,house_size_sqm,no_of_rooms,no_of_bathrooms,large_living_room,parking_space,front_garden,swimming_pool,distance_to_school_km,wall_fence,house_age_or_renovated,water_front,distance_to_supermarket_km,property_value
4949,297,261.0,4,4,1.0,1,1,1,2.9,1,20,0,4.1,254547
4950,302,282.0,4,2,1.0,1,1,1,3.8,1,12,1,4.8,299788
4951,153,147.0,3,4,1.0,1,0,0,3.3,1,7,0,4.7,138088
4952,180,158.0,2,1,0.0,1,0,1,2.2,0,1,0,3.4,151907
4953,695,619.0,8,6,1.0,1,1,1,0.4,1,5,0,0.4,566249


In [9]:
data.isna().sum()

land_size_sqm                  0
house_size_sqm                 1
no_of_rooms                    0
no_of_bathrooms                0
large_living_room              2
parking_space                  0
front_garden                   0
swimming_pool                  0
distance_to_school_km         12
wall_fence                     0
house_age_or_renovated         0
water_front                    0
distance_to_supermarket_km    38
property_value                 0
dtype: int64

In [10]:
data["distance_to_school_km"] = data["distance_to_school_km"].fillna(data["distance_to_school_km"].mean())
data["distance_to_supermarket_km"] = data["distance_to_supermarket_km"].fillna(data["distance_to_supermarket_km"].mean())
data["house_size_sqm"] = data["house_size_sqm"].fillna(data["house_size_sqm"].mean())
data["large_living_room"] = data["large_living_room"].fillna(data["large_living_room"].mean())

In [11]:
data.isna().sum()

land_size_sqm                 0
house_size_sqm                0
no_of_rooms                   0
no_of_bathrooms               0
large_living_room             0
parking_space                 0
front_garden                  0
swimming_pool                 0
distance_to_school_km         0
wall_fence                    0
house_age_or_renovated        0
water_front                   0
distance_to_supermarket_km    0
property_value                0
dtype: int64

In [12]:
X=data.drop(["property_value"],axis=1)
y=data["property_value"]

In [16]:
# Define the input features and target variable
X_train = data.drop([ "property_value"], axis=1)
y_train = data["property_value"]
X_test  = data.drop([ "property_value"], axis=1)
y_test  = data["property_value"]

In [17]:
data.columns

Index(['land_size_sqm', 'house_size_sqm', 'no_of_rooms', 'no_of_bathrooms',
       'large_living_room', 'parking_space', 'front_garden', 'swimming_pool',
       'distance_to_school_km', 'wall_fence', 'house_age_or_renovated',
       'water_front', 'distance_to_supermarket_km', 'property_value'],
      dtype='object')

In [18]:
# Define a function to calculate the mean squared error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

# Define a function to split the data based on a given feature and threshold value
def split_data(X, y, feature, threshold):
    left_mask = X[feature] <= threshold
    right_mask = X[feature] > threshold
    X_left, y_left = X.loc[left_mask], y.loc[left_mask]
    X_right, y_right = X.loc[right_mask], y.loc[right_mask]
    return X_left, y_left, X_right, y_right

# Define a function to find the best split point for a given feature
def find_best_split(X, y, feature):
    # Sort the data by the feature
    X_sorted = X.sort_values(feature)
    y_sorted = y[X_sorted.index]
    # Initialize the best split point and lowest mse
    best_split = None
    lowest_mse = float("inf")
    # Loop through each unique value of the feature
    for threshold in X_sorted[feature].unique():
        # Split the data at the current threshold
        X_left, y_left, X_right, y_right = split_data(X_sorted, y_sorted, feature, threshold)
        # Calculate the mse for the current split
        mse = mean_squared_error(y_left, y_left.mean()) + mean_squared_error(y_right, y_right.mean())
        # Update the best split and lowest mse if the current mse is lower
        if mse < lowest_mse:
            best_split = (feature, threshold)
            lowest_mse = mse
    return best_split, lowest_mse

In [23]:
class Node:
    def __init__(self, X, y, depth=0, max_depth=5, min_samples_split=2):
        self.X = X
        self.y = y
        self.depth = depth
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.left = None
        self.right = None
        self.split_feature = None
        self.split_threshold = None
        self.prediction = y.mean()
        # Stop splitting if the node reaches the maximum depth or minimum number of samples
        if depth < max_depth and len(X) >= min_samples_split:
            # Find the best split point among all features
            best_split, lowest_mse = min([find_best_split(X, y, feature) for feature in X.columns], key=lambda x: x[1])
            # Create left and right child nodes and continue splitting recursively
            if best_split is not None:
                self.split_feature, self.split_threshold = best_split
                X_left, y_left, X_right, y_right = split_data(X, y, self.split_feature, self.split_threshold)
                self.left = Node(X_left, y_left, depth+1, max_depth, min_samples_split)
                self.right = Node(X_right, y_right, depth+1, max_depth, min_samples_split)
                self.prediction = None

    def predict_row(self, row):
        # Recursively traverse the tree to find the prediction for a given row
        if self.prediction is not None:
            return self.prediction
        elif row[self.split_feature] <= self.split_threshold:
            return self.left.predict_row(row)
        else:
            return self.right.predict_row(row)

    def predict(self, X):
        # Make predictions for all rows in X
        return np.array([self.predict_row(X.iloc[i]) for i in range(len(X))])

# Train the decision tree on the training data
tree = Node(X_train, y_train)

# Make predictions on the testing data
y_pred = tree.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 7050538107.769412


In [25]:
y_pred

array([223653.65542819, 223653.65542819, 223653.65542819, ...,
       223653.65542819, 223653.65542819, 223653.65542819])

In [26]:
pred_df = pd.DataFrame({'price':y_pred})

In [27]:
pred_df.head()

,price
0,223653.655428
1,223653.655428
2,223653.655428
3,223653.655428
4,223653.655428


In [28]:
def rmse(y_test, y_pred):
    mse = np.mean((y_test - y_pred)**2)
    rmse = np.sqrt(mse)
    return rmse

In [29]:
# Make predictions on the testing data
y_pred = tree.predict(X_test)

# Calculate the root mean squared error
rmse_value = rmse(y_test, y_pred)

# Print the RMSE value
print("RMSE:", rmse_value)

RMSE: 83967.48244272548
